In [1]:
from dendrocat import RadioSource
from astropy.io import fits
from spectral_cube import SpectralCube
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import numpy as np
import sys
import importlib
from astropy.wcs import WCS
from scipy import interpolate
import pandas as pd
import sympy as syp
from astropy import units as u
import radio_beam
from astropy.table import Table
from radio_beam import Beam
from astropy import coordinates
from astropy import wcs 
from astropy.nddata.utils import Cutout2D
from dendrocat.aperture import Ellipse
from regions import Regions, PixCoord
from astropy import stats
from itertools import chain
from radio_beam import Beams
from astropy.convolution import convolve
import astropy.constants as c



W51 = '/orange/adamginsburg/w51/TaehwaYoo/'
W51b6 = '/orange/adamginsburg/w51/TaehwaYoo/2015.1.01596.S_W51_B6_LB/continuum_images/'
W51cont='/orange/adamginsburg/w51/TaehwaYoo/b6contfits/'
w51e2_b6_briggs=W51cont+'W51e2_cont_bigbriggs.image.fits'
w51e2_b6_robust0=W51cont+'W51e2_cont_big_robust0.image.fits'
w51e2_b6_uniform=W51cont+'W51e2_cont_biguniform.image.fits'
w51e2_b6_superuniform=W51cont+'W51e2_cont_bigsuperuniform.image.fits'

w51n_b6_briggs = W51cont+'W51n_cont_bigbriggs.image.fits'
w51n_b6_robust0 = W51cont+'w51n_cont_big_robust0.image.fits'
w51n_b6_uniform = W51cont+'W51n_cont_biguniform.image.fits'
w51n_b6_superuniform = W51cont+'W51n_cont_bigsuperuniform.image.fits'
w51n_b6_natural = W51cont+'W51n_cont_bignatural.image.fits'

W51b3 = '/orange/adamginsburg/w51/TaehwaYoo/2017.1.00293.S_W51_B3_LB/may2021_successful_imaging/'

w51n_b3_tt0 = W51b3+'w51n.spw0thru19.14500.robust0.thr0.075mJy.mfs.I.startmod.selfcal7.image.tt0.pbcor.fits'
w51n_b3_tt1 = W51+'w51n.spw0thru19.14500.robust0.thr0.075mJy.mfs.I.startmod.selfcal7.image.tt1.pbcor.fits'
w51n_b3_alpha = W51+'w51n.spw0thru19.14500.robust0.thr0.075mJy.mfs.I.startmod.selfcal7.alpha.pbcor.fits'

w51conv = '/orange/adamginsburg/w51/TaehwaYoo/convolved_new/'
w51n_b6_conv = w51conv + 'w51n_cont_bigbriggs.image.convB3_briggs.fits'

w51e_b3_tt0 = W51b3+'w51e2.spw0thru19.14500.robust0.thr0.075mJy.mfs.I.startmod.selfcal7.image.tt0.pbcor.fits'
w51e2_b3_tt1 = W51+'w51e2.spw0thru19.14500.robust0.thr0.075mJy.mfs.I.startmod.selfcal7.image.tt1.pbcor.fits'
w51e2_b3_alpha = W51+'w51e2.spw0thru19.14500.robust0.thr0.075mJy.mfs.I.startmod.selfcal7.alpha.pbcor.fits'

w51e2_b6_conv = w51conv + 'w51e2_cont_bigbriggs.image.convB3_briggs.fits'

w51e_b6_almaimf_conv = '/orange/adamginsburg/w51/TaehwaYoo/w51_alma_imf/w51e_B6_conv.fits'
w51e_b3_almaimf_conv = '/orange/adamginsburg/w51/TaehwaYoo/w51_alma_imf/w51e_B3_conv.fits'


w51n_b3_almaimf_conv = '/orange/adamginsburg/w51/TaehwaYoo/w51_alma_imf/w51n_B3_conv.fits'

w51n_b6_almaimf = '/orange/adamginsburg/w51/TaehwaYoo/w51_alma_imf/W51-IRS2_B6_uid___A001_X1296_X187_continuum_merged_12M_robust0_selfcal9_finaliter.image.tt0.pbcor.fits'

w51n_b6_conv = w51conv + 'w51n_new_nocorr_in_area_B6_conv.fits'
#w51n_b3_conv = w51conv + 'w51n_B3_conv.fits'
#w51e_b3_conv = w51conv + 'w51e_B3_conv.fits'
w51e_b6_conv = w51conv + 'w51e_new_nocorr_in_area_B6_conv.fits'


w51e_matched_catalog = '/home/t.yoo/w51/catalogue/dendrogram/dendro_w51e_matched.fits'
w51n_matched_catalog = '/home/t.yoo/w51/catalogue/dendrogram/dendro_w51n_matched.fits'

w51e_b3_noiseregion = '/orange/adamginsburg/w51/TaehwaYoo/w51e_b3_std_sky.reg'
w51e_b6_noiseregion = '/orange/adamginsburg/w51/TaehwaYoo/w51e_b6_std_sky.reg'
w51n_b3_noiseregion = '/orange/adamginsburg/w51/TaehwaYoo/w51n_b3_std_sky.reg'
w51n_b6_noiseregion = '/orange/adamginsburg/w51/TaehwaYoo/w51n_b6_std_sky.reg'
w51e_b6_calibrated_pbcor = '/orange/adamginsburg/w51/TaehwaYoo/w51e2.spw0thru19.14500.robust0.thr0.15mJy.mfs.I.startmod.selfcal7.image.tt0.pbcor.fits'
w51n_b6_calibrated_pbcor = '/orange/adamginsburg/w51/TaehwaYoo/w51n.spw0thru19.14500.robust0.thr0.1mJy.mfs.I.startmod.selfcal7.image.tt0.pbcor.fits'


photometrydir = '/home/t.yoo/w51/catalogue/photometry/'
w51e_b3_flux = photometrydir+'w51e_b3_flux_size.fits'
w51n_b3_flux = photometrydir+'w51n_b3_flux_size.fits'
w51e_b6_flux = photometrydir+'w51e_b6_flux_size.fits'
w51n_b6_flux = photometrydir+'w51n_b6_flux_size.fits'
w51e_b6_conv_flux = photometrydir+'w51e_b6_conv_flux_size.fits'
w51n_b6_conv_flux = photometrydir+'w51n_b6_conv_flux_size.fits'







In [2]:
def cen_freq(center, delta ,alpha=2):
    start = center-delta/2 ; end = center+delta/2
    freqarr = np.logspace(np.log10(start),np.log10(end),20)
    dfreq = freqarr[1:]-freqarr[:-1]
    dfreq = np.append(dfreq,[dfreq[-1]])
    
    integral_up = np.sum(freqarr**(alpha+1)*dfreq)
    integral_down = np.sum(freqarr**(alpha)*dfreq)
    return integral_up/integral_down

def get_noise(image, wcsNB, noiseregion):
    container = []
    for reg in noiseregion:
        pix_reg = reg.to_pixel(wcsNB)
        noisemask = pix_reg.to_mask()
        noiseim = noisemask.cutout(image)
        container.append(noiseim.flatten())
    noiseim = list(chain.from_iterable(container))
    std = stats.mad_std(noiseim,ignore_nan=True)

    return std

def get_alpha(b3flux, b6flux, b3freq, b6freq):
    
    alpha = np.log(b3flux/b6flux)/np.log(b3freq/b6freq)
    return alpha
 
def flux_gaussian_multiple_apertures(flux, major, minor, rarr): # major, minor in FWHM, au unit
    flux_rescaled = flux*major/minor # making a 2d symmetric Gaussian with FWHM = FWHM_major
    major_sigma = major/np.sqrt(8*np.log(2))
    peak_height = flux / 2 / np.pi/ major/ minor * 8 * np.log(2)
    flux_rarr =2*np.pi*major_sigma**2 * peak_height * (1 - np.exp(-rarr**2/2/major_sigma**2))
    return flux_rarr


    
    

In [3]:
import matplotlib as mpl
plt.rcParams['axes.labelsize']=25
plt.rcParams['xtick.labelsize']=25
plt.rcParams['ytick.labelsize']=25
plt.rcParams['axes.titlesize']=25
mpl.rcParams['axes.linewidth'] = 5
mpl.rcParams['xtick.major.size'] = 10
mpl.rcParams['xtick.major.width'] = 4
mpl.rcParams['xtick.minor.size'] = 5
mpl.rcParams['xtick.minor.width'] = 2
mpl.rcParams['ytick.major.size'] = 10
mpl.rcParams['ytick.major.width'] = 4
mpl.rcParams['ytick.minor.size'] = 5
mpl.rcParams['ytick.minor.width'] = 2
params = {"xtick.top": True, "ytick.right": True, "xtick.direction": "in", "ytick.direction": "in"}

rarr = [100,200,300,400,500]

w51e_b3_flux_fits = Table.read(w51e_b3_flux)
w51e_b6_flux_fits = Table.read(w51e_b6_flux)
w51n_b3_flux_fits = Table.read(w51n_b3_flux)
w51n_b6_flux_fits = Table.read(w51n_b6_flux)
w51e_b6_flux_conv_fits = Table.read(w51e_b6_conv_flux)
w51n_b6_flux_conv_fits = Table.read(w51n_b6_conv_flux)

w51e_b3_major = w51e_b3_flux_fits['deconvolved_major'].value* np.pi/180 * 5.41*1000*206265
w51e_b6_major = w51e_b6_flux_fits['deconvolved_major'].value* np.pi/180 * 5.41*1000*206265
w51n_b3_major = w51n_b3_flux_fits['deconvolved_major'].value* np.pi/180 * 5.1*1000*206265
w51n_b6_major = w51n_b6_flux_fits['deconvolved_major'].value* np.pi/180 * 5.1*1000*206265
w51e_b3_minor = w51e_b3_flux_fits['deconvolved_minor'].value* np.pi/180 * 5.41*1000*206265
w51e_b6_minor = w51e_b6_flux_fits['deconvolved_minor'].value* np.pi/180 * 5.41*1000*206265
w51n_b3_minor = w51n_b3_flux_fits['deconvolved_minor'].value* np.pi/180 * 5.1*1000*206265
w51n_b6_minor = w51n_b6_flux_fits['deconvolved_minor'].value* np.pi/180 * 5.1*1000*206265


w51e_b3_int_flux = w51e_b3_flux_fits['flux']
w51e_b6_int_flux = w51e_b6_flux_conv_fits['flux']
w51n_b3_int_flux = w51n_b3_flux_fits['flux']
w51n_b6_int_flux = w51n_b6_flux_conv_fits['flux']

w51e_flux_b3 = flux_gaussian_multiple_apertures(w51e_b3_int_flux[i],w51e_b3_major[i], w51e_b3_minor[i], rarr)
w51e_flux_b6 = flux_gaussian_multiple_apertures(w51e_b6_int_flux[i],w51e_b6_major[i], w51e_b6_minor[i], rarr)
w51n_flux_b3 = flux_gaussian_multiple_apertures(w51n_b3_int_flux[i],w51n_b3_major[i], w51n_b3_minor[i], rarr)
w51n_flux_b6 = flux_gaussian_multiple_apertures(w51n_b6_int_flux[i],w51n_b6_major[i], w51n_b6_minor[i], rarr)

freqb3=92982346121.91989*u.Hz
freqb6=226691598706.70853*u.Hz

w51e_alpha = np.log(w51e_flux_b3/w51e_flux_b6)/np.log(freqb3/freqb6)
w51n_alpha = np.log(w51n_flux_b3/w51n_flux_b6)/np.log(freqb3/freqb6)

alpha_w51n_err = np.sqrt((b3noise/b3flux)**2+(b6noise/b6flux)**2)/np.log(freqb3/freqb6)



1.83594537278e-05 deg
convolution /orange/adamginsburg/w51/TaehwaYoo/w51e2.spw0thru19.14500.robust0.thr0.15mJy.mfs.I.startmod.selfcal7.image.tt0.pbcor.fits
unmatched 32 33
noise 2.065537678896079e-05 0.00012361802520203196
[ 72  73  74  75  76  77  78  79  80  81  82 116 117 118 119 120 121 122
 123 124 125 126 127 128 129 130 131 132 133 134 135 136]
92982346121.91989 226691598706.70853
[ 2.96625047e+00  2.34316242e+00  2.45506891e+00  3.26472797e+00
  2.83448206e+00  2.51378980e+00  2.50396449e+00  7.08080930e-01
  2.18696003e+00  2.30766893e+00  2.27147317e+00  2.70056116e+00
  1.87316470e+00  2.20489432e+00  2.82170333e+00  2.37793999e+00
  2.67892596e+00  2.25207420e+00  3.26727790e+00  2.33118975e+00
  2.09986950e+00  2.70863872e+00  2.76226671e+00  1.97934017e+00
  2.40317847e+00  1.87257230e+00  2.97391542e+00  2.52749973e+00
  1.88155608e+00  1.08795240e+00  2.99619544e+00  2.52075538e+00
  2.83578788e+00  1.38731761e+00  2.02129117e+00  2.41279821e+00
  2.82331076e+00  1.8957

NameError: name 'freqb3' is not defined

In [ ]:
b6image = convolveb3b6(w51n_b3_tt0,w51n_b6_calibrated_pbcor,'/orange/adamginsburg/w51/TaehwaYoo/convolved_new/','w51n_new_nocorr_in_area')

alpha_w51n,b3flux, b6flux, b3noise, b6noise = measure_alpha(w51n_b3_tt0, w51n_b6_conv,read_w51n_b3_noise,read_w51n_b6_noise, w51n_matched_catalog,b6image=b6image)
alpha_w51n_err = np.sqrt((b3noise/b3flux)**2+(b6noise/b6flux)**2)/np.log(freqb3/freqb6)




In [ ]:
import matplotlib as mpl
plt.rcParams['axes.labelsize']=25
plt.rcParams['xtick.labelsize']=25
plt.rcParams['ytick.labelsize']=25
plt.rcParams['axes.titlesize']=25
mpl.rcParams['axes.linewidth'] = 5
mpl.rcParams['xtick.major.size'] = 10
mpl.rcParams['xtick.major.width'] = 4
mpl.rcParams['xtick.minor.size'] = 5
mpl.rcParams['xtick.minor.width'] = 2
mpl.rcParams['ytick.major.size'] = 10
mpl.rcParams['ytick.major.width'] = 4
mpl.rcParams['ytick.minor.size'] = 5
mpl.rcParams['ytick.minor.width'] = 2
params = {"xtick.top": True, "ytick.right": True, "xtick.direction": "in", "ytick.direction": "in"}
plt.rcParams.update(params)
w51ecat = Table.read(w51e_matched_catalog)
w51e_matched = np.where(w51ecat['is_matched'])[0]
w51ncat = Table.read(w51n_matched_catalog)
w51n_matched = np.where(w51ncat['is_matched'])[0]
w51e_unmatched_b3 = np.where(w51ecat['b3_xsky']<0)[0]
w51e_unmatched_b6 = np.where(w51ecat['b6_xsky']<0)[0]
w51n_unmatched_b3 = np.where(w51ncat['b3_xsky']<0)[0]
w51n_unmatched_b6 = np.where(w51ncat['b6_xsky']<0)[0]
fig = plt.figure(figsize=(8,16))
ax1 = fig.add_axes([0.15,0.58,0.8,0.4])
ax2 = fig.add_axes([0.15,0.08,0.8,0.4])

bins = np.linspace(-1.5,4.5,31)
ax1.hist(alpha_w51e[w51e_matched], bins=bins, facecolor='none',edgecolor='b', histtype='step',label='W51-E matched',lw=4)
ax1.hist(alpha_w51e[w51e_unmatched_b3], bins=bins, alpha=0.3, color='cyan', label='W51-E B3 unmatched',hatch='o')
ax1.hist(alpha_w51e[w51e_unmatched_b6], bins=bins, alpha=0.3, color='cyan', label='W51-E B6 unmatched',hatch='////')


ax1.hist(alpha_w51n[w51n_matched], bins=bins, facecolor='none',color='peru', histtype='step',label='W51-IRS2 matched',lw=4)
ax1.hist(alpha_w51n[w51n_unmatched_b3], bins=bins, alpha=0.3, color='orange', label='W51-IRS2 B3 unmatched',hatch='o')
ax1.hist(alpha_w51n[w51n_unmatched_b6], bins=bins, alpha=0.3, color='orange', label='W51-IRS2 B6 unmatched',hatch='////')

ax1.set_xlabel('spectral indices')
ax1.set_ylabel('YSO number')
ax1.legend(fontsize=15, frameon=False,loc='upper left')
ax1.set_xticks([-1,0,1,2,3,4])
ax1.set_yticks([0,2,4,6,8,10,12,14,16,18])



ind1_w51e = np.where(alpha_w51e[w51e_matched]-3*alpha_w51e_err[w51e_matched]>1.7)[0]
ind2_w51e = np.where(alpha_w51e[w51e_matched]+3*alpha_w51e_err[w51e_matched]<1.7)[0]
ax2.hist(alpha_w51e[w51e_matched][ind1_w51e], bins=bins, alpha=0.8, edgecolor='b', histtype='step',label=r'W51-E $\alpha>2$ ($3\sigma$)',lw=4,ls='dotted')
ax2.hist(alpha_w51e[w51e_matched][ind2_w51e], bins=bins, alpha=0.8, edgecolor='b', histtype='step',label=r'W51-E $\alpha<2$ ($3\sigma$)',lw=4)
ind1_w51n = np.where(alpha_w51n[w51n_matched]-3*alpha_w51n_err[w51n_matched]>1.7)[0]
ind2_w51n = np.where(alpha_w51n[w51n_matched]+3*alpha_w51n_err[w51n_matched]<1.7)[0]
ax2.hist(alpha_w51n[w51n_matched][ind1_w51n], bins=bins, alpha=0.8, edgecolor='orange', histtype='step',label=r'W51-IRS2 $\alpha>2$ ($3\sigma$)',lw=4,ls='dotted')
ax2.hist(alpha_w51n[w51n_matched][ind2_w51n], bins=bins, alpha=0.8, edgecolor='orange', histtype='step',label=r'W51-IRS2 $\alpha<2$ ($3\sigma$)',lw=4)
ax2.set_xlabel('spectral indices')
ax2.set_ylabel('YSO number')
ax2.legend(fontsize=15, frameon=False,loc='upper left')
ax2.set_xticks([-1,0,1,2,3,4])
ax2.set_yticks([0,2,4,6,8,10,12,14,16,18])
plt.savefig('spectral_indices.png')

In [ ]:
HIIind_w51e = np.where((alpha_w51e<0.5)&(w51ecat['is_matched']))[0]

HIIind_w51n = np.where((alpha_w51n<0.5)&(w51ncat['is_matched']))[0]

print(HIIind_w51e,HIIind_w51n )